In [384]:
import numpy as np
from math import log10, exp, sqrt
from scipy.stats import binom
from copy import deepcopy

def generate_state_key_names(state_index):
    state_letters = ['U', 'D']
        
    if state_index == 0:
        return ['State 0']
    else:
        state_digits = int(log10(state_index))+1
        state_range = range(1, state_index + 1)
        state_pattern_U = "{{0:0{0:d}d}}".format(state_digits)
        state_pattern_D = "{{1:0{0:d}d}}".format(state_digits)
        state_pattern = "State U{0:s}-D{1:s}".format(state_pattern_U, state_pattern_D)
        state_labels = [state_pattern.format(state_index - num_d, num_d) for num_d in range(state_index+1)]
        
        return state_labels

In [294]:
class tree_trunk:
    def __init__(self, N_steps, state_names):
        # Constructor
        
        # Define some self-facts
        self.N_steps = N_steps
        self.state_names = state_names
        
        # Period range
        period_numbers = range(N_steps + 1)
        period_names = ['Period %d' % x for x in period_numbers]
        
        # Initialize dict
        self.trunk = dict.fromkeys(period_names)
        
        # Create state-level dict
        state_dict = dict.fromkeys(state_names)
        
        # Fill states
        trunk_keys = list(self.trunk.keys())
        for pnum in period_numbers:
            state_keys = generate_state_key_names(pnum)
            self.trunk[trunk_keys[pnum]] = dict.fromkeys(state_keys)
            for state_key in self.trunk[trunk_keys[pnum]].keys():
                self.trunk[trunk_keys[pnum]][state_key] = state_dict.copy()
            
    def print_leaf(self, step_index, leaf_index):
        # Printing of information about single leaf
        trunk_keys = list(self.trunk.keys())
        target_key = trunk_keys[step_index]
        branch_keys = list(self.trunk[target_key].keys())
        target_branch_key = branch_keys[leaf_index]
        
        print("In {0:s}".format(target_branch_key))
        print(self.trunk[target_key][target_branch_key])
            
    
    def print(self, step_indexes):
        # Reasonably pretty print of tree leafs at selected steps
        trunk_keys = list(self.trunk.keys())
        for step_index in step_indexes:
            target_key = trunk_keys[step_index]
            print("--------------------------------------------------------------------")
            print("In Tree At {0:s}:".format(target_key))
            print("--------------------------------------------------------------------")
            for state_idx in range(step_index+1):
                self.print_leaf(step_index, state_idx)
                print("\n")

In [295]:
zz = tree_trunk(3, ["Stock", "RN Probability"])

In [296]:
zz.print([0, 1])

--------------------------------------------------------------------
In Tree At Period 0:
--------------------------------------------------------------------
In State 0
{'Stock': None, 'RN Probability': None}


--------------------------------------------------------------------
In Tree At Period 1:
--------------------------------------------------------------------
In State U1-D0
{'Stock': None, 'RN Probability': None}


In State U0-D1
{'Stock': None, 'RN Probability': None}




In [458]:
class binomial_tree:
    def __init__(self, Stock_0, N_steps, h_time_step_length, r_int_rate, d_div_rate, s_volatility):
        
        # Save model parameters to self
        self.N_steps = N_steps
        self.h_time_step_length = h_time_step_length
        self.r_int_rate = r_int_rate
        self.d_div_rate = d_div_rate
        self.s_volatility = s_volatility
        
        # Calculate up- and down- factors as in Slide 40 of Deck 2 (forward tree)
        u_fctr = exp((r_int_rate - d_div_rate) * h_time_step_length + s_volatility * sqrt(h_time_step_length))
        d_fctr = exp((r_int_rate - d_div_rate) * h_time_step_length - s_volatility * sqrt(h_time_step_length))
        
        self.u_fctr = u_fctr
        self.d_fctr = d_fctr
        
        # Calculate RN probablity
        rn_probability = exp((r_int_rate - d_div_rate) * h_time_step_length) - d_fctr
        rn_probability = rn_probability / (u_fctr - d_fctr)
        
        self.rn_pstar = rn_probability
        
        # Tree state names
        state_names = ["Stock", "Probability of State", "Multi-Period Discount Factor", "Single-Period Discount Factor"]
        
        # Create tree trunk
        self.tree = tree_trunk(N_steps, state_names)
        
        ## Fill tree trunk
        # Multi-Period Discount Factors
        discount_factors = np.linspace(N_steps, 0, N_steps + 1)
        discount_factors = np.exp(- discount_factors * r_int_rate * h_time_step_length)
        
        self.discount_factors = discount_factors
        
        time_index = 0
        for time_key in self.tree.trunk.keys():
            loc_df = discount_factors[time_index]
            time_index = time_index + 1
            for state_key in self.tree.trunk[time_key].keys():
                self.tree.trunk[time_key][state_key]["Multi-Period Discount Factor"] = loc_df
                self.tree.trunk[time_key][state_key]["Single-Period Discount Factor"] = discount_factors[-2]
                
        # Save Multi-Period DFs to DataFrame
        
        # Risk-neutral probabilities of states
        time_index = 0
        for time_key in self.tree.trunk.keys():
            loc_probs = binom.pmf(range(time_index+1), time_index, rn_probability)
            time_index = time_index + 1
            state_index = 0
            for state_key in self.tree.trunk[time_key].keys():
                self.tree.trunk[time_key][state_key]["Probability of State"] = loc_probs[state_index]
                state_index = state_index + 1
                
        # Save RN probabilities to DataFrame
        
        # Stock price values in states
        time_index = 0
        for time_key in self.tree.trunk.keys():
            ud_fctrs = [(u_fctr ** (time_index - num_d)) * (d_fctr ** num_d) for num_d in range(time_index+1)]
            time_index = time_index + 1
            state_index = 0
            for state_key in self.tree.trunk[time_key].keys():
                self.tree.trunk[time_key][state_key]["Stock"] = Stock_0 * ud_fctrs[state_index]
                state_index = state_index + 1
        
        # Save stock prices to DataFrame
        
    def copy(self):
        return deepcopy(self)
        
    def pricing(self, derivative_product):
        
        # Create a copy of own dictionary
        derivative_tree = deepcopy(self.tree)
        
        # Go through the tree in reverse
        pricing_start_periods = list(derivative_tree.trunk.keys())[:-1]
        pricing_start_periods.reverse()
        pricing_end_periods = list(derivative_tree.trunk.keys())[1:]
        pricing_end_periods.reverse()
        
        for kk in pricing_end_periods:
            print(kk)
        

In [459]:
zz = binomial_tree(100, 4, 0.25, 0.02, 0.01, 0.4)

In [460]:
class derivative:
    def __init__(self, payoff_function, exercise_type = "European"):
        self.payoff_function = payoff_function
        self.exercise_type = exercise_type
        self.pricing_tree = None

In [461]:
zz.pricing(a)

['Period 3', 'Period 2', 'Period 1', 'Period 0']
Period 4
Period 3
Period 2
Period 1
